<a href="https://colab.research.google.com/github/ShubhamPrakash108/DogVsCat_Classifier/blob/main/EDA_IBM_Course.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Link:- https://www.kaggle.com/competitions/playground-series-s4e7/code

In [18]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'playground-series-s4e7:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F73291%2F8930475%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240724%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240724T084340Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D474db68bd0d22ad8d9447f43ae708725c752382b91d32ac58c4709e646df0940b98c2ee4a1fd4c53afbd1b26d31e9b88d20f82717924571a10d2dcb98f437febe47ff0c00b8b68bc393d97a907fce20cdaabd1fd06ef659a127fc878ce9c00a7d9cc8df2893786a63c636c924ed384f961757931e039ec11a129f698fb65f520d61fa4551037591e65cd87d4bfd320f2e7c350886341bf75f7da68d757b6b4dc9eea5645dc53793c7c38ff1c550b15cebd5acedf6ad3ae0d5c73d2da52f4b37ddc99276162d40dd6d718298753888b52e27ccbc26e6c47ef8c91ccf22477c9e49248757090fa50eff6b9e3fd3cc2b9f59531dfe176ea0ade060b3d219ab9f5ce'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 259402906 bytes downloaded
Downloaded and uncompressed: playground-series-s4e7
Data source import complete.


In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e7/sample_submission.csv
/kaggle/input/playground-series-s4e7/test.csv
/kaggle/input/playground-series-s4e7/train.csv


###Imports

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import scipy
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [21]:
data = pd.read_csv("/kaggle/input/playground-series-s4e7/train.csv")
data.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0


In [22]:
#There is no use of Id for this problem
data.drop('id', axis=1, inplace=True)

##EDA

#####Data Cleaning

Cleaning the data is important:
Good data = Good results.
I am going to identify duplicate or unnecessary data and I will also be dealing with the outliers.

How can data be messy?
1. Duplicate and Missing data
2. Unnecessary data, inconsistent text and typos
3. Outliers( it skews the data)



Policies for Missing data
1. When we talk about removing the data as a method for handling missing values, it means that we choose to eliminate the entire row of data that contains the missing value. This approach is often used when we believe that the missing values are random and do not carry any significant information.

Here are the key points to consider when removing the data:

Pros:

Quick and straightforward: Removing the entire row with missing values can quickly clean up the dataset without the need to guess or estimate replacement values.
Minimal impact: If the missing values are only present in a small number of rows, removing those rows may not have a significant impact on the overall dataset.
Cons:

Loss of information: By removing rows with missing values, we may lose valuable information that could be important for our analysis or modeling.
Potential bias: If certain columns have missing values for many rows, removing those rows may introduce bias into the dataset, as the missing values may be related to a specific reason or condition.
It's important to carefully consider the pros and cons of removing data with missing values in order to make an informed decision. If the missing values are minimal and randomly distributed, removing the data may be a viable option. However, if the missing values are substantial or potentially carry important information, alternative methods such as imputation or masking may be more appropriate.

2. Imputing the data is another method for handling missing values in a dataset. Instead of removing the rows with missing values, we replace those missing values with estimated values based on the available data.

Here are the key points to understand about imputing the data:

Pros:

Retains information: Imputing the missing values allows us to keep the entire dataset intact, including the rows and columns that may contain valuable information for our analysis or modeling.
Preserves sample size: By imputing the missing values, we can maintain the original sample size, which can be important for statistical analysis and modeling.
Flexibility in estimation: We can choose different methods for imputing the missing values, such as using the mean, median, or more complex estimation techniques. This flexibility allows us to tailor the imputation method to the specific characteristics of the dataset.
Cons:

Introduction of uncertainty: Imputing missing values introduces an additional level of uncertainty, as the imputed values are estimates rather than actual observed values. This uncertainty can affect the accuracy and reliability of our analysis or modeling.
Potential bias: Depending on the imputation method used, there is a risk of introducing bias into the dataset. The imputed values may not accurately represent the true values of the missing data, which can impact the validity of our results.

3. Mask the data: Masking the data is another approach for handling missing values in a dataset. Instead of removing or imputing the missing values, we treat them as a separate category or indicator, indicating that the value is missing.

Here are the key points to understand about masking the data:

Pros:

Retains information: Masking the missing values allows us to keep the entire dataset intact, including the rows and columns that may contain valuable information for our analysis or modeling.
Preserves sample size: By masking the missing values, we can maintain the original sample size, which can be important for statistical analysis and modeling.
Potential insights: Treating missing values as a separate category assumes that the missingness itself carries useful information. In some cases, missing values may indicate patterns or reasons that can provide insights into the data.
Cons:

Introduction of uncertainty: Masking the missing values introduces an additional level of uncertainty, as we are treating the missing values as a distinct category. This uncertainty can affect the accuracy and reliability of our analysis or modeling.
Assumption of similarity: Treating all missing values as alike assumes that they have a common reason for being missing. This assumption may not always hold true, and it can lead to biased results if the missing values are not truly similar.
When considering masking the data, it is important to carefully evaluate whether missing values carry meaningful information and whether treating them as a separate category aligns with the goals of the analysis. This approach can be useful when missing values are not extensive and when there is a reasonable assumption that the missingness itself is informative.





Outliers: Outliers are observations in a dataset that significantly deviate from the majority of the data points. These observations are distinct and can have a substantial impact on the analysis or modeling process. Here are the key points to understand about outliers:

Definition: Outliers are data points that are located far away from the central tendency of the dataset. They can be unusually high or low values compared to the rest of the data.
Impact on analysis: Outliers can have a significant impact on statistical analysis and modeling. They can skew the results, affect the accuracy of predictions, and distort the overall patterns and relationships in the data.
Identification: Outliers can be identified through various methods, such as visual inspection of plots like histograms, density plots, and box plots. They can also be detected using mathematical calculations, such as percentiles and interquartile range.
Interpretation: It is important to interpret outliers carefully. While some outliers may be data errors or anomalies that need to be addressed, others may provide valuable insights into the data. Outliers can indicate rare events, extreme conditions, or unusual patterns that are worth investigating further.
Handling outliers: The approach to handling outliers depends on the specific context and goals of the analysis. In some cases, outliers may be removed if they are deemed as data errors. However, it is crucial to exercise caution when removing outliers, as they may contain valuable information. Alternatively, outliers can be transformed or adjusted to reduce their impact on the analysis.
Overall, outliers should be carefully examined and considered in the analysis process. They can provide valuable insights or introduce biases, depending on the context. It is important to strike a balance between addressing outliers that are detrimental to the analysis and preserving outliers that carry meaningful information.


________________________________________________________________________________
Residuals are the differences between the actual values and the predicted values of a model. They represent the model's failure to accurately predict the outcome. Residuals are an important concept in exploratory data analysis for machine learning because they can be used to detect outliers and assess the performance of the model.

There are different approaches to leverage residuals for outlier detection:

Standardized Residuals: These are obtained by dividing the residual by the standard error. Standardizing the residuals allows for comparison across different outcome variable ranges.

Deleted Residuals: In this approach, an observation is removed from the dataset, and the model is retrained without that observation. The difference between the original model's prediction and the prediction with the deleted observation can help identify outliers.

Studentized Residuals: Similar to deleted residuals, studentized residuals involve removing one observation at a time and standardizing the residuals based on the range of the model. This approach helps assess the impact of each observation on the model.
Once an outlier is detected using residuals, there are several ways to handle it:

Remove the outlier altogether: This eliminates the outlier's effect on the model, but it may result in the loss of important information.
Assign a different value to the outlier: By replacing the outlier with a different value, you can mitigate its impact on the model while retaining the rest of the row.
Transform the column with the outlier: Applying a transformation, such as a log transformation, to the column containing the outlier may change its range and make it no longer an outlier.
Predict the value of the outlier: This can be done by using similar observations or regression techniques to estimate what the value would have been if it were not an outlier.
Keep the outlier: If you are using a model that is resistant to outliers, you may choose to keep the outlier and account for its influence in the analysis.


In [23]:
# Getting the overview of the data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11504798 entries, 0 to 11504797
Data columns (total 11 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Gender                object 
 1   Age                   int64  
 2   Driving_License       int64  
 3   Region_Code           float64
 4   Previously_Insured    int64  
 5   Vehicle_Age           object 
 6   Vehicle_Damage        object 
 7   Annual_Premium        float64
 8   Policy_Sales_Channel  float64
 9   Vintage               int64  
 10  Response              int64  
dtypes: float64(3), int64(5), object(3)
memory usage: 965.5+ MB


In [24]:
# Getting the standard mathematical insights of the numerical data
for col in data.columns:
  if data[col].dtype != 'object':
    print(data[col].describe())

count    1.150480e+07
mean     3.838356e+01
std      1.499346e+01
min      2.000000e+01
25%      2.400000e+01
50%      3.600000e+01
75%      4.900000e+01
max      8.500000e+01
Name: Age, dtype: float64
count    1.150480e+07
mean     9.980220e-01
std      4.443120e-02
min      0.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: Driving_License, dtype: float64
count    1.150480e+07
mean     2.641869e+01
std      1.299159e+01
min      0.000000e+00
25%      1.500000e+01
50%      2.800000e+01
75%      3.500000e+01
max      5.200000e+01
Name: Region_Code, dtype: float64
count    1.150480e+07
mean     4.629966e-01
std      4.986289e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: Previously_Insured, dtype: float64
count    1.150480e+07
mean     3.046137e+04
std      1.645475e+04
min      2.630000e+03
25%      2.527700e+04
50%      3.182400e+04
75%      3.945100e+04
max

In [25]:
# Checking the categorical columns in the dataset
for col in data.columns:
  if data[col].value_counts().size < 7 :
    print(f"{col} : {data[col].value_counts()}")

Gender : Gender
Male      6228134
Female    5276664
Name: count, dtype: int64
Driving_License : Driving_License
1    11482041
0       22757
Name: count, dtype: int64
Previously_Insured : Previously_Insured
0    6178116
1    5326682
Name: count, dtype: int64
Vehicle_Age : Vehicle_Age
1-2 Year     5982678
< 1 Year     5044145
> 2 Years     477975
Name: count, dtype: int64
Vehicle_Damage : Vehicle_Damage
Yes    5783229
No     5721569
Name: count, dtype: int64
Response : Response
0    10089739
1     1415059
Name: count, dtype: int64


In [26]:
data.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0


In [27]:
#Finding the correlation of Response with others
data_num = data.select_dtypes(include = ['float64', 'int64'])
data_response = data_num.corr()['Response'][:-1]
top_features = data_response[abs(data_response) > 0.3].sort_values(ascending=False)
print("There is {} strongly correlated values with SalePrice:\n{}".format(len(top_features), top_features))

There is 1 strongly correlated values with SalePrice:
Previously_Insured   -0.34593
Name: Response, dtype: float64


In [ ]:
#Generating the correlation graph using seaborn
for i in range(0, len(data.columns), 5):
    sns.pairplot(data=data,
                x_vars=data.columns[i:i+5],
                y_vars=['Response'])

###########Edit
Looking for Correlations¶
Before proceeding with the data cleaning, it is useful to establish a correlation between the response variable (in our case the sale price) and other predictor variables, as some of them might not have any major impact in determining the price of the house and will not be used in the analysis. There are many ways to discover correlation between the target variable and the rest of the features. Building pair plots, scatter plots, heat maps, and a correlation matrixes are the most common ones. Below, we will use the corr() function to list the top features based on the pearson correlation coefficient (measures how closely two sequences of numbers are correlated). Correlation coefficient can only be calculated on the numerical attributes (floats and integers), therefore, only the numeric attributes will be selected.



The range of skewness for a fairly symmetrical bell curve distribution is between -0.5 and 0.5; moderate skewness is -0.5 to -1.0 and 0.5 to 1.0; and highly skewed distribution is < -1.0 and > 1.0. In our case, we have ~1.7, so it is considered highly skewed data.

Now, we can try to transform our data, so it looks more normally distributed. We can use the np.log() function from the numpy library to perform log transform. This documentation contains more information about the numpy log transform.


In [ ]:
# Checking wether the data is normally distributed or not by using Log Transform
sp_untransformed = sns.displot(data['Age'], height=5, aspect=1.5)

In [ ]:
print("Skewness: %f" % data['Age'].skew())

Now, we can try to transform our data, so it looks more normally distributed. We can use the np.log() function from the numpy library to perform log transform. This documentation contains more information about the numpy log transform.

In [ ]:
log_transformed = np.log(data['Age'])
sp_transformed = sns.displot(log_transformed)
print("Skewness: %f" % (log_transformed).skew())

In [ ]:
#Lets see whether there are some duplicates or not
data.duplicated().sum()

Finding the Missing Values
For easier detection of missing values, pandas provides the isna(), isnull(), and notna() functions. For more information on pandas missing values please check out this documentation.

To summarize all the missing values in our dataset, we will use isnull() function. Then, we will add them all up, by using sum() function, sort them with sort_values() function, and plot the first 20 columns (as the majority of our missing values fall within first 20 columns), using the bar plot function from the matplotlib library.

In [ ]:
#Handling the missing values(if any)
total = data.isnull().sum().sort_values(ascending=False)
total_select = total.head(20)
total_select.plot(kind="bar", figsize = (8,6), fontsize = 10)

plt.xlabel("Columns", fontsize = 20)
plt.ylabel("Count", fontsize = 20)
plt.title("Total Missing Values", fontsize = 20)

So, there is no missing values

There are several options for dealing with missing values. We will use 'Lot Frontage' feature to analyze for missing values.

We can drop the missing values, using dropna() method.
housing.dropna(subset=["Lot Frontage"])
Using this method, all the rows, containing null values in 'Lot Frontage' feature, for example, will be dropped.

We can drop the whole attribute (column), that contains missing values, using the drop() method.
housing.drop("Lot Frontage", axis=1)
Using this method, the entire column containing the null values will be dropped.

We can replace the missing values (zero, the mean, the median, etc.), using fillna() method.
median = housing["Lot Frontage"].median()
median
housing["Lot Frontage"].fillna(median, inplace = True)





_______________________________________________________________________________________

Feature Scaling One of the most important transformations we need to apply to our data is feature scaling. There are two common ways to get all attributes to have the same scale: min-max scaling and standardization.

Min-max scaling (or normalization) is the simplest: values are shifted and rescaled so they end up ranging from 0 to 1. This is done by subtracting the min value and dividing by the max minus min.

Standardization is different: first it subtracts the mean value (so standardized values always have a zero mean), and then it divides by the standard deviation, so that the resulting distribution has unit variance.

Scikit-learn library provides MinMaxScaler for normalization and StandardScaler for standardization needs. For more information on scikit-learn MinMaxScaler and StandardScaler please visit their respective documentation websites.

First, we will normalize our data.

norm_data = MinMaxScaler().fit_transform(hous_num) norm_data Note the data is now a ndarray

we can also standardize our data.

In [ ]:
#do after changing the data into numbers
norm_data = MinMaxScaler().fit_transform(data["Age"])
norm_data

Handling the Outliers¶
Finding the Outliers
In statistics, an outlier is an observation point that is distant from other observations. An outlier can be due to some mistakes in data collection or recording, or due to natural high variability of data points. How to treat an outlier highly depends on our data or the type of analysis to be performed. Outliers can markedly affect our models and can be a valuable source of information, providing us insights about specific behaviours.

There are many ways to discover outliers in our data. We can do Uni-variate analysis (using one variable analysis) or Multi-variate analysis (using two or more variables). One of the simplest ways to detect an outlier is to inspect the data visually, by making box plots or scatter plots.

Uni-variate Analysis
A box plot is a method for graphically depicting groups of numerical data through their quartiles. Box plots may also have lines extending vertically from the boxes (whiskers) indicating variability outside the upper and lower quartiles. Outliers may be plotted as individual points. To learn more about box plots please click here.

In [ ]:
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

data['Normalized_Age'] = MinMaxScaler().fit_transform(data[['Age']])

sns.boxplot(data=data.drop(['Region_Code'], axis=1), x='Normalized_Age', orient='h')

As we can see from these two plots, we have some points that are plotted outside the box plot area and that greatly deviate from the rest of the population. Whether to remove or keep them will greatly depend on the understanding of our data and the type of analysis to be performed. In this case, the points that are outside of our box plots in the 'Lot Area' and the 'Sale Price' might be the actual true data points and do not need to be removed.

Bi-variate Analysis
Next, we will look at the bi-variate analysis of the two features, the sale price, 'SalePrice', and the ground living area, 'GrLivArea', and plot the scatter plot of the relationship between these two parameters.

In [ ]:
price_area = data.plot.scatter(x='Annual_Premium',
                      y='Age')

Sure! Let's dive into how to use various data visualization libraries in Python, specifically focusing on Matplotlib, Pandas, and Seaborn. We will go through some practical examples with code snippets to illustrate their usage.

### Data Visualization Libraries in Python

1. **Matplotlib**:
   - A foundational plotting library in Python.
   - Highly customizable with a variety of plotting options.

2. **Pandas**:
   - Offers convenient plotting capabilities through a wrapper around Matplotlib.
   - Less flexible than Matplotlib but easier and quicker for simple plots.

3. **Seaborn**:
   - Built on top of Matplotlib.
   - Provides high-level interfaces for drawing attractive statistical graphics.

### Example 1: Scatter Plot with Matplotlib

```python
import matplotlib.pyplot as plt

# Let's assume we have a DataFrame 'df' with columns 'sepal_length' and 'sepal_width'.
df = ...  # Your DataFrame here

# Basic scatter plot
plt.scatter(df['sepal_length'], df['sepal_width'], marker='o')
plt.xlabel('Sepal Length')
plt.ylabel('Sepal Width')
plt.title('Scatter Plot of Sepal Length vs Sepal Width')
plt.show()
```

### Example 2: Multiple Layers in Scatter Plot

```python
# Adding another layer to the scatter plot
plt.scatter(df['sepal_length'], df['sepal_width'], marker='o', label='Sepal')
plt.scatter(df['petal_length'], df['petal_width'], marker='x', label='Petal')
plt.xlabel('Length')
plt.ylabel('Width')
plt.title('Scatter Plot of Sepal and Petal Dimensions')
plt.legend()
plt.show()
```

### Example 3: Histogram with Matplotlib

```python
# Histogram of 'sepal_length'
plt.hist(df['sepal_length'], bins=20)
plt.xlabel('Sepal Length')
plt.ylabel('Frequency')
plt.title('Histogram of Sepal Length')
plt.show()
```

### Example 4: Object-Oriented Interface in Matplotlib

```python
fig, ax = plt.subplots()
ax.bar(df['sepal_length'][:10], df['sepal_width'][:10])
ax.set_xlabel('Sepal Length')
ax.set_ylabel('Sepal Width')
ax.set_title('Bar Plot of Sepal Dimensions')
plt.show()
```

### Example 5: Plotting with Pandas

```python
# Group by 'species' and plot mean values
grouped = df.groupby('species').mean()
grouped.plot(kind='line', figsize=(8, 6), fontsize=10, colormap='viridis')
plt.xlabel('Species')
plt.ylabel('Mean Values')
plt.title('Mean Sepal and Petal Dimensions by Species')
plt.show()
```

### Example 6: Pair Plot with Seaborn

```python
import seaborn as sns

# Pair plot to show relationships between all pairs of features
sns.pairplot(df, hue='species', height=2.5)
plt.show()
```

### Example 7: Hexbin Plot with Seaborn

```python
# Joint plot with hexbin
sns.jointplot(x='sepal_length', y='sepal_width', data=df, kind='hex', gridsize=20)
plt.show()
```

### Example 8: Facet Grid with Seaborn

```python
# Facet grid to plot histograms for each species
g = sns.FacetGrid(df, col='species')
g.map(plt.hist, 'sepal_length')
plt.show()
```

### Recap

- **Matplotlib**: Highly flexible, powerful for detailed customization.
- **Pandas**: Convenient for quick plots directly from DataFrames.
- **Seaborn**: Great for statistically informative and visually appealing plots.

In practice, you might start with Pandas for quick insights, use Matplotlib for detailed and custom visualizations, and leverage Seaborn for advanced statistical graphics.

In [ ]:
data.head()

In [ ]:
plt.rcParams['agg.path.chunksize'] = 10000

# Now try your plotting code again, for example:
plt.plot(data['Age'], data['Response'])
plt.show()

Here is a summary of the codes discussed:

Grouping and Plotting with Pandas:

Grouping the data frame by a specific column using the groupby() function.
Calculating the mean for each feature using the mean() function.
Plotting the grouped data using the plot() function.
Setting line style to blank to remove lines from the plot.
Setting specific colors for each feature using the color parameter.
Setting font size and figure size using the fontsize and figsize parameters.
Pair Plotting with Seaborn:

Importing the Seaborn library using import seaborn as sns.
Using the pairplot() function to create scatter plots between each feature.
Setting the hue parameter to break down the plot by color.
Setting the size parameter to define the size of the plot.
Visualizing correlations and histograms across all columns.
Hex Bin Plotting with Seaborn:

Using the jointplot() function to create a hex bin plot.
Specifying the x and y values for the plot.
Showing the density of similar points using hexagons.
Displaying histograms of the x and y values.
Facet Grid Plotting with Seaborn:
Using the FacetGrid() function to create a grid of subplots.
Specifying the data and column arguments.
Setting margin_titles to True to add titles to each subplot.
Mapping a histogram to each subplot using the map() function.
These codes demonstrate how to perform exploratory data analysis and create various visualizations using Pandas and Seaborn libraries.

In [ ]:
sns.pairplot(data, hue='Response',height=3)

We can use .str.replace('Iris','') to replace the string 'Iris' with an empty string. Lambda function can also be used.

Feature Engineering:  Feature engineering involves encoding categorical data and transforming variables to ensure a linear relationship. We will also discuss feature scaling, which is important for many machine learning algorithms. The section will provide a foundation for understanding linear regression and its assumptions. Overall, this section will help you prepare your data for modeling and improve the performance of your machine learning algorithms.



Revise : Polynomial Features [link](https://www.coursera.org/learn/ibm-exploratory-data-analysis-for-machine-learning/lecture/K4120/variable-transformation)


Here's a summary of the key points about variable selection and transformation in data modeling:

1. Variable selection involves choosing which features to include in a model.

2. Variables often need transformation before being included in models:
   - Log and polynomial transformations
   - Encoding (converting non-numeric features to numeric)
   - Scaling (adjusting numeric data to comparable scales)

3. Encoding methods depend on the feature type:
   - For nominal data (no inherent order):
     - Binary encoding (for two-value variables)
     - One-hot encoding (for multiple-value variables)
   - For ordinal data (inherent order):
     - Ordinal encoding

4. Binary encoding:
   - Converts variables to 0 or 1
   - Suitable for true/false or two-option variables

5. One-hot encoding:
   - Creates multiple binary columns, one for each category
   - Used for variables with multiple possible values

6. Ordinal encoding:
   - Converts ordered categories to numerical values
   - Usually uses integers (e.g., low=1, medium=2, high=3)
   - Consider if the distance between categories is meaningful

The choice of encoding method depends on the data type and the importance of preserving order in categorical variables. Practitioners should consider these factors when preparing data for modeling.

Feature Scaling : Variable selection involves choosing the set of features to include in a model, often requiring transformation before inclusion. These transformations include:

Log and Polynomial Transformations:

Used to adjust the distribution and relationship of data.
Encoding:

Converts non-numeric features (categorical or ordinal) to numeric features.
Binary Encoding: Converts variables with two possible values to 0 or 1 (e.g., male/female).
One-Hot Encoding: Converts variables with multiple values into multiple binary columns. For example, a "color" column with values "red," "blue," and "green" is converted into three columns: "red," "blue," and "green," with binary indicators.
Ordinal Encoding: Converts ordered categories into numerical values (e.g., "low," "medium," "high" to 1, 2, 3). This assigns a set distance between categories, which may not always be appropriate.
Scaling:

Adjusts the scale of numeric data to make features comparable.
The appropriate method of encoding and scaling depends on the type of feature. For example, nominal data (without inherent order) like colors are suited for one-hot encoding, while ordinal data (with inherent order) like temperature levels can be ordinally encoded.

Encoding Methods:
Binary Encoding:
Suitable for variables with two possible values.
Example: Gender (male/female), Married (yes/no).
One-Hot Encoding:
Suitable for variables with multiple categories.
Example: Color (red/blue/green).
Ordinal Encoding:
Suitable for ordered categories.
Example: Satisfaction level (low/medium/high).
Key Points:
Log and polynomial transformations help in adjusting data distribution.
Encoding is crucial for transforming categorical data to numeric.
Scaling ensures numeric data is on a comparable scale.
The choice of encoding method depends on whether the categorical data has an inherent order or not.

Transformations Based on Feature Type
Continuous Numerical Values:

Transformations: Standard Scaling, Min-Max Scaling, and Robust Scaling.
Python Implementation:
Use StandardScaler for standard scaling.
Use MinMaxScaler for Min-Max scaling.
Use RobustScaler for robust scaling.
Nominal or Categorical Data (Unordered):

Example: True/False, Red/Blue/Green.
Transformations:
Binary encoding for features with only two values.
One-hot encoding for features with multiple values.
Python Implementation:
Use LabelEncoder, LabelBinarizer, and OneHotEncoder from sklearn.
Use get_dummies from pandas for one-hot encoding.
Ordinal Data (Ordered Categorical Data):

Example: Low/Medium/High.
Transformations:
Encode with numbers representing the order (e.g., 1, 2, 3).
Python Implementation:
Use DictVectorizer to assign numbers to different values.
Use OrdinalEncoder to pass in the names of values in the correct order.

________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

Inferential statistics and hypothesis testing are two types of data analysis often overlooked at early stages of analyzing your data. They can give you quick insights about the quality of your data. They also help you confirm business intuition and help you prescribe what to analyze next using Machine Learning. This module looks at useful definitions and simple examples that will help you get started creating hypothesis around your business problem and how to test them.

Learning Objectives

Understand statistical estimation and inference


Recognize parametric and non-parametric approaches to modeling


Become familiarized with the most common statistical distributions


Fundamental understanding of frequentist vs. Bayesian statistics


Hands-on experience communicating insights to peers and stakeholders


Hands-on experience providing insights from data


Hands-on experience assessing the quality of data


Hands-on experience on Exploratory Data Analysis

  This paragraph introduces a section on basic statistical concepts that are important for machine learning and data-driven decision making. Here's a breakdown of the key points:

Learning goals for the section:

Understanding estimation vs. inference in statistics
Differentiating between parametric and non-parametric modeling approaches
Exploring common statistical distributions
Introducing frequentist vs. Bayesian statistics


Estimation vs. Inference:

Estimation: Provides an estimate of a parameter (e.g., mean) from sample data
Inference: Aims to understand the underlying population distribution, including estimates and other parameters like standard error


Relationship between machine learning and statistical inference:

Both use sample data to infer qualities of the underlying population distribution
Machine learning may focus on the entire distribution or specific features
Some ML models emphasize understanding underlying parameters, while others focus more on prediction results


Business example: Customer churn prediction

Target variable: Whether a customer has left the company
Features: Customer tenure, purchase history, age, location, etc.
Prediction: Probability of a customer leaving (e.g., 0.99 likely to leave, 0.01 likely to stay)


Estimation vs. Inference in the churn example:

Estimation: Calculating the impact of each feature (e.g., each additional year as a customer reduces churn by 20%)
Inference: Providing statistical significance and confidence intervals for estimates (e.g., 95% confidence interval of 19-21% vs. -10% to 50%)



The paragraph emphasizes the importance of understanding these statistical concepts for effective machine learning and data analysis in business contexts.

Estimation and Inference:

Here's a summary of the key points from the customer churn data analysis, including relevant code snippets:

1. Dataset: Telco customer churn dataset from IBM Cognos Analytics
   - Stored in a Pandas DataFrame called `df_phone`

2. Bar plot of churn value by payment type:
```python
import seaborn as sns
import matplotlib.pyplot as plt

sns.barplot(x='payment', y='churn_value', data=df_phone, ci=None)
plt.show()
```

3. Bar plot of churn value by tenure (months):
```python
import pandas as pd

df_phone['tenure_bins'] = pd.cut(df_phone['tenure_months'], bins=5)
sns.barplot(x='tenure_bins', y='churn_value', data=df_phone, ci=None)
plt.show()
```

4. Pair plot for multiple features:
```python
selected_features = ['tenure_months', 'gb_used_per_month', 'total_revenue', 'cltv', 'churn_value']
sns.pairplot(df_phone[selected_features], hue='churn_value')
plt.show()
```

5. Hexbin plot for tenure vs. monthly charge:
```python
sns.jointplot(x='tenure_months', y='monthly_charge', data=df_phone, kind='hex')
plt.show()
```

Key observations:
- Credit card users are less likely to churn compared to bank withdrawal or mailed check users.
- Customers with shorter tenures are more likely to churn.
- The pair plot shows relationships between multiple variables and their distributions, split by churn status.
- The hexbin plot reveals density patterns in the relationship between tenure and monthly charges.

These visualizations provide insights into factors affecting customer churn, which can be used for further analysis and decision-making in customer retention strategies.

Estimation and Inference - Parametric vs. Non-Parametric:(REVISE)

This text discusses parametric and non-parametric statistical models, their characteristics, and common probability distributions. Here's a summary of the key points:

1. Parametric vs. Non-parametric Models:
   - Parametric models have a finite number of parameters and make strict assumptions about data distributions.
   - Non-parametric models rely on fewer assumptions and are distribution-free.

2. Examples:
   - Non-parametric: Histograms for creating data distributions
   - Parametric: Normal distribution with mean and standard deviation parameters

3. Maximum Likelihood Estimation (MLE):
   - Common method for estimating parameters in parametric models
   - Maximizes the likelihood function based on observed data

4. Common Probability Distributions:
   - Uniform: Equal probability for all values in a range
   - Normal (Gaussian): Bell-shaped curve, defined by mean and standard deviation
   - Log-normal: Log transformation yields a normal distribution
   - Exponential: Models time between events
   - Poisson: Models number of events in a fixed time interval

5. Central Limit Theorem:
   - Averages of multiple samples tend to follow a normal distribution

6. Real-world applications:
   - Customer Lifetime Value estimation
   - Height distributions
   - Income distributions
   - Video viewing patterns

The text emphasizes the importance of understanding these concepts for statistical modeling and data analysis in various fields.




In [ ]:
sns.barplot(x='Response', y='Annual_Premium', data=data)
plt.ylabel("Annual_Premium")
plt.xlabel("Response")
plt.title("Response vs Annual_Premium")

#Final

In [ ]:
data.shape

In [ ]:
data_ohe = pd.get_dummies(data, columns=['Gender','Vehicle_Age','Vehicle_Damage'])

In [ ]:
data_ohe.shape

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

X = data_ohe.drop('Response', axis=1)
y = data_ohe['Response']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("Training Features:\n", X_train)
print("Test Features:\n", X_test)
print("Training Target:\n", y_train)
print("Test Target:\n", y_test)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Logistic Regression Accuracy: {accuracy:.4f}')


In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Create and train the model
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Make predictions and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Decision Tree Accuracy: {accuracy:.4f}')


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create and train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Make predictions and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Random Forest Accuracy: {accuracy:.4f}')


In [ ]:
!pip install xgboost
import xgboost as xgb

# Create and train the model
model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss')
model.fit(X_train, y_train)

# Make predictions and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'XGBoost Accuracy: {accuracy:.4f}')


In [ ]:
from sklearn.svm import SVC

# Create and train the model
model = SVC()
model.fit(X_train, y_train)

# Make predictions and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'SVM Accuracy: {accuracy:.4f}')


In [ ]:
!pip install psutil
import psutil

# Number of physical cores
physical_cores = psutil.cpu_count(logical=False)

# Number of logical cores
logical_cores = psutil.cpu_count(logical=True)

# CPU frequency
cpu_freq = psutil.cpu_freq()

# CPU utilization percentage
cpu_percent = psutil.cpu_percent(interval=1)

# Print CPU specifications
print(f"Physical cores: {physical_cores}")
print(f"Logical cores: {logical_cores}")
print(f"Max Frequency: {cpu_freq.max:.2f}Mhz")
print(f"Min Frequency: {cpu_freq.min:.2f}Mhz")
print(f"Current Frequency: {cpu_freq.current:.2f}Mhz")
print(f"CPU Utilization: {cpu_percent}%")

# Per-core utilization
print("Per-core utilization:")
for i, percentage in enumerate(psutil.cpu_percent(percpu=True, interval=1)):
    print(f"Core {i}: {percentage}%")
